# Hangyakolónia algoritmus – gráfos gyakorló feladatok (sablon)

Ez a notebook **gyakorló feladatokhoz** készült.  
Ahol `TODO` jelölést látsz, ott neked kell megírni a kódot.

## 1. feladat (könnyű): Gráf reprezentáció és szomszédok

Reprezentáld a gráfot, majd írj `neighbors(node)` függvényt.

In [ ]:
# TODO: gráf létrehozása
graph = {
    # 0: {1: 2.0, 2: 5.0},
    # ...
}

def neighbors(node):
    """Add vissza a node szomszédait és él-súlyait."""
    # TODO
    raise NotImplementedError

## 2. feladat (könnyű): Heurisztika η_ij

Definiáld a heurisztikus értéket az élekre, pl. η_ij = 1 / distance.

In [ ]:
# TODO: heurisztika
def heuristic_eta(i, j, graph):
    # TODO
    raise NotImplementedError

## 3. feladat (közepes): Lépési valószínűségek

Számítsd ki p_ij^k értékeit a dián szereplő képlet alapján.

In [ ]:
# TODO: lépési valószínűségek
def transition_probabilities(i, tabu, tau, graph, alpha=0.5):
    """Visszaadja a {j: p_ij} szótárat."""
    # TODO
    raise NotImplementedError

## 4. feladat (közepes): Egy hangya útja

Építs fel egy hangya útvonalat start csúcsból, tabu listával.

In [ ]:
import random

# TODO: egy hangya megoldása
def construct_ant_solution(start_node, graph, tau, alpha=0.5):
    # TODO
    raise NotImplementedError

## 5. feladat (közepes): Feromonfrissítés

Használd a τ_ij(t) = τ_ij(t-1) + Δτ_ij jellegű frissítést.

In [ ]:
# TODO: feromonfrissítés
def update_pheromones_for_path(tau, path, Q=1.0):
    # TODO
    raise NotImplementedError

## 6. feladat (nehéz): Egyszerű ACO ciklus

Rakd össze egy egyszerű ACO fő ciklust több hangyával.

In [ ]:
# TODO: egyszerű ACO futtatása
def run_simple_aco(graph, n_ants=5, n_iters=20, alpha=0.5):
    # TODO
    raise NotImplementedError